In [2]:
#!pip install bs4
#!pip install selenium
#!pip install openpyxl

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1279 sha256=fd8b245269f634573a7663ad26098bc8897e22c56f2832147f2e2d1dd271c074
  Stored in directory: c:\users\s510\appdata\local\pip\cache\wheels\0a\9e\ba\20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


In [17]:
from selenium import webdriver
import time
from random import uniform
from datetime import datetime, timedelta
import time
from bs4 import BeautifulSoup
import openpyxl
from openpyxl.styles import PatternFill, Color
from openpyxl import Workbook
from random import *
import re
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait

In [27]:
# chrome driver 패스 설정
path = "C:/chromedriver_win32/chromedriver.exe"
driver = webdriver.Chrome(path)

In [16]:
path = "C:/geckodriver-v0.26.0-win64/geckodriver.exe"
driver = webdriver.Firefox(executable_path=path)

In [18]:
profile = webdriver.FirefoxProfile()
profile.set_preference('general.useragent.override', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:65.0) Gecko/20100101 Firefox/65.0')
profile.set_preference("network.proxy.type", 1)
profile.set_preference("network.proxy.socks", "127.0.0.1")
profile.set_preference("network.proxy.socks_port", 9050)

path = "C:/geckodriver-v0.26.0-win64/geckodriver.exe"
driver = webdriver.Firefox(firefox_profile=profile, executable_path=path)

In [44]:
page_index = 1
# 크롤링 시작 일자
f = '2020.01.17'
# 크롤링 종료 일자
t = '2020.01.31'

In [45]:
# 네이버 지식인 크롤링
# keyword에 크롤링하고 싶은 단어 선택. space 는 + 로 치환
def get_keyword(text):
    return text.replace(" ", "%20")

In [46]:
# 정렬 방식 선택
# 1: 추천순
# 2: 최신순
# 기타: 정확도 순
def sort_kind(index):
    # 추천
    if index == 1:
        return 'vcount'
    # 최신순
    elif index == 2:
        return 'date'
    # 정확도
    else:
        return 'none'

In [47]:
keyword = '코로나'
driver.get('https://kin.naver.com/search/list.nhn?query=' + get_keyword(keyword))
time.sleep(uniform(0.1, 1.0))

In [48]:
period_txt = "&period=" + f + ".%7C" + t + "."

_sort_kind = sort_kind(2)
date = str(datetime.now()).replace('.', '_')
date = date.replace(' ', '_')
date = date.replace(':', ';')

# URL 저장
f = open("result/url_list" + "_" + keyword.replace(' ', '+') + "_" + date + ".txt", 'w')
page_url = []
while True:
    time.sleep(uniform(0.01, 1.0))
    driver.implicitly_wait(10)
    driver.get('https://kin.naver.com/search/list.nhn?' + "&sort=" + _sort_kind + '&query=' + get_keyword(keyword) + period_txt + "&section=kin" + "&page=" + str(page_index))
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    tags = soup.find_all('a', class_="_nclicks:kin.txt _searchListTitleAnchor")
    for tag in tags:
        url = str(tag).split(' ')[3]
        url = url.replace('href=', "")
        url = url.replace('"', "")
        url = url.replace('amp;', '')
        page_url.append(url)
        f.write(url + "\n")

    post_number = driver.find_element_by_class_name('number').text
    post_number = str(post_number).replace("(", "")
    post_number = str(post_number).replace(")", "")
    
    current_number = post_number.split('/')[0].split('-')[1]
    current_number = current_number.replace(',', '')
    total_number = post_number.split('/')[1]
    total_number = total_number.replace(',', '')

    if int(current_number) == int(total_number):
        break
    else:
        page_index += 1

filename = 'result/' + keyword.replace(' ', '.') + "_" + date + "_crawling_result.xlsx"


In [49]:
count =0

In [50]:
def clean_text(text):
        mod_text = re.sub('[가-힣]','',text)
        return mod_text

In [51]:
wb = Workbook()
sheet = wb.active
sheet.append(['번호','제목', '질문','질문날짜','답변자정보', '답변','답변날짜'])

for j in range(1, 8):
    sheet.cell(row=1, column=j).fill = PatternFill(start_color='808080', end_color='808080', fill_type='solid')

for i in page_url:
    driver.implicitly_wait(10)
    driver.get(i)
    title = driver.find_element_by_class_name('title').text
    try:
        question_txt = driver.find_element_by_class_name('c-heading__content').text        
            
    except:
        question_txt = ""
        
    question_date = driver.find_elements_by_class_name('c-userinfo__info')
    #question_date = clean_text(question_date).strip()
    answer_job = driver.find_elements_by_class_name('c-heading-answer__title')        
    answer_date = driver.find_elements_by_class_name('c-heading-answer__content-date')

    # 답변 리스트
    answer_list = driver.find_elements_by_class_name("se-main-container")
    #answer_list = driver.find_elements_by_class_name("se-section.se-section-text.se-l-default")    
    
    for n, answer in enumerate(answer_list):
        
        texts = answer.find_elements_by_tag_name('p')
        t = ""     
        
        for i in texts:             
            
            t += i.text
            
        if n == 0:
            #qd = clean_text(question_date[n].text)
            qd = clean_text(question_date[n].text.strip())
            sheet.append([count,title, question_txt,qd,answer_job[n].text, t,answer_date[n].text])
        else:
            #print(n)
            sheet.append(["","","", "",answer_job[n].text, t,answer_date[n].text])
    
    count += 1
    print(count)
    if(count % 100 == 0):
        time.sleep(20)
        if(count % 1000 == 0):
            time.sleep(100)
    wb.save(filename)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


UnexpectedAlertPresentException: Alert Text: 오류가 발생했습니다. 잠시 후 다시 시도해주세요.
Message: unexpected alert open: {Alert text : 오류가 발생했습니다. 잠시 후 다시 시도해주세요.}
  (Session info: chrome=83.0.4103.97)


In [111]:
ex_list